In [97]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")


import pandas as pd
import numpy as np
from src.network_plots import write_main_verses_from_dict, get_centrality_types_stats_plot_crosstext
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import cm



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
holy_texts = ["Bible_NT", "Quran", "Torah", "Gita", "Analects"]

encoder_models = ['jina_clip_v1', 'all_MiniLM_L6_v2']

parameters = ["05_threshold", "075_threshold"]

for encoder_model in encoder_models:

    for parameter in parameters:

        analytics_data = {}

        for text in holy_texts:
            with open("../data/analytics_data/"+ encoder_model +"/"+ parameter+ "/"+text+"_network_results.pkl", "rb") as f:
                analytics_data_ = pickle.load(f)
                analytics_data[text] = analytics_data_

        analytics_data_stats = {}

        for text in holy_texts:
            with open("../data/analytics_data/"+ encoder_model +"/"+ parameter+ "/"+text+"_network_stats_results.pkl", "rb") as f:
                analytics_data_ = pickle.load(f)
                analytics_data_stats[text] = analytics_data_


        average_clustering_coefficients =  {text: analytics_data[text]["Average Clustering Coefficient"] for text in holy_texts}
        average_path_lengths = {text: analytics_data[text]["Average Path Length"] for text in holy_texts}

        average_degree_centralitys = {text: analytics_data[text]["Average Degree Centrality"] for text in holy_texts}
        average_eigenvector_centralitys = {text: analytics_data[text]["Average Eigenvector Centrality"] for text in holy_texts}
        average_betweenness_centralitys = {text: analytics_data[text]["Average Betweenness Centrality"] for text in holy_texts}
        average_closeness_centralitys = {text: analytics_data[text]["Average Closeness Centrality"] for text in holy_texts}

        std_degree_centralitys = {text: analytics_data_stats[text]["Degree Centrality Std"] for text in holy_texts}
        std_eigenvector_centralitys = {text: analytics_data_stats[text]["Eigenvector Centrality Std"] for text in holy_texts}
        std_betweenness_centralitys = {text: analytics_data_stats[text]["Betweenness Centrality Std"] for text in holy_texts}
        std_closeness_centralitys = {text: analytics_data_stats[text]["Closeness Centrality Std"] for text in holy_texts}

        dictionary_list = [average_clustering_coefficients,average_path_lengths,average_degree_centralitys, average_eigenvector_centralitys,
                            average_betweenness_centralitys,average_closeness_centralitys, std_degree_centralitys, std_eigenvector_centralitys, std_betweenness_centralitys, std_closeness_centralitys]

        # Convert to DataFrame with each dictionary as a column
        df = pd.DataFrame.from_dict(dictionary_list).T

        # Assign new column names if desired
        df.columns = ['average_clustering_coefficients', 'average_path_lengths', 'average_degree_centralitys', 'average_eigenvector_centralitys', 'average_betweenness_centralitys', 'average_closeness_centralitys', 'std_degree_centralitys', 'std_eigenvector_centralitys', 'std_betweenness_centralitys', 'std_closeness_centralitys']

        sns.set_style("whitegrid")

        get_centrality_types_stats_plot_crosstext(parameter, encoder_model, df)

/Users/joao/RAGenesis/notebooks/../src/network_plots.py:552: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yscale('log')
/Users/joao/RAGenesis/notebooks/../src/network_plots.py:552: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yscale('log')
/Users/joao/RAGenesis/notebooks/../src/network_plots.py:552: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yscale('log')
/Users/joao/RAGenesis/notebooks/../src/network_plots.py:552: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yscale('log')
/Users/joao/RAGenesis/notebooks/../src/network_plots.py:567: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  filepath = "../

In [95]:


def get_centrality_types_stats_plot_crosstext(parameter, encoder_model, df):

    df_transposed = df.transpose()

    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))  # Create a 2x2 grid of subplots
    sns.set_style("whitegrid")


    # List of column names to plot, assuming they are in your DataFrame
    centrality_types = ['degree', 'eigenvector', 'betweenness', 'closeness']  # Adjust these column names as necessary

    for ax, column in zip(axes.flat, centrality_types):

        # num_bars = df_transposed.shape[0]
        
        colormap = cm.viridis(np.linspace(0, 0.7, 5))

        # Plot selected column with Viridis colors
        bars = df_transposed.loc['average_'+column +'_centralitys'].plot(
            kind='bar',
            yerr=df_transposed.loc['std_'+ column+'_centralitys'],
            capsize=4,
            color=colormap,
            edgecolor='black',
            linewidth=0.7,
            ax=ax  # Specify the current axis
        )

        # Customize each subplot
        ax.set_title(f" {column.capitalize()} Centrality ", fontsize=14, fontweight='bold')
        ax.set_xlabel("Texts", fontsize=11)
        ax.set_ylabel("Average Value", fontsize=11)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=9)
        ax.set_yticklabels(ax.get_yticks(), fontsize=9)
        ax.grid(axis='y', linestyle='--', alpha=0.7)
        ax.set_yscale('log')

        # Remove top and right spines
        sns.despine(ax=ax, left=True, bottom=True)

        # Add data labels
        # for container in bars.containers:
        #     ax.bar_label(container, fmt='%.2f', label_type='edge', fontsize=8, padding=2)

    # Adjust layout
    plt.tight_layout()

    # Show the plot
    plt.show()

    filepath = "../data/analytics_data/"+ encoder_model +"/"+ parameter+ "/crosstext/centrality_comparison.png"
        
    plt.savefig(filepath)

/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_985/4289747874.py:39: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(ax.get_yticks(), fontsize=9)
/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_985/4289747874.py:39: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(ax.get_yticks(), fontsize=9)
/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_985/4289747874.py:39: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(ax.get_yticks(), fontsize=9)
/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_985/4289747874.py:39: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_ytick

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import seaborn as sns



# Assume the first column is the statistics type and index by it
df_transposed = df.set_index('Unnamed: 0').transpose()

# Plotting
plt.figure(figsize=(12, 8))

# Set a seaborn style for a cleaner look
sns.set_style("whitegrid")

# Get the number of bars to plot
num_bars = df_transposed.shape[0]

# Create a colormap
colormap = cm.viridis(np.linspace(0, 0.7, 4))

# Plot means with Viridis colors
bars = df_transposed['mean'].plot(
    kind='bar',
    yerr=df_transposed['std'],
    capsize=4,
    color=colormap,
    edgecolor='black',
    linewidth=0.7
)


# Customize the plot further
plt.title("Summary Statistics For Verse Token Length", fontsize=16, fontweight='bold')
plt.xlabel("Texts", fontsize=13)
plt.ylabel("Token Length", fontsize=13)
plt.xticks(rotation=45, fontsize=11)
plt.yticks(fontsize=11)

# Add gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Remove top and right spines for a cleaner look
sns.despine(left=True, bottom=True)

# Add a legend with a better placement
# plt.legend(["Mean"], fontsize=12, loc='upper right')

# Add data labels on top of the bars
for container in bars.containers:
    bars.bar_label(container, fmt='%.2f', label_type='edge', fontsize=10, padding=3)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'holy_texts' is a list of categories and 'metrics' and 'titles' are defined
categories = holy_texts

for metric, title in zip(metrics, titles):
    # Set a style
    sns.set_theme(style="whitegrid")

    # Create barplot with different colors
    plt.figure(figsize=(8, 6))
    sns.barplot(x=categories, y=metric, palette="viridis")  # You can choose other palettes like "pastel", "deep", etc.

    # Enhance the appearance
    plt.title(title, fontsize=16, weight='bold')
    plt.xlabel('Category', fontsize=14, weight='bold')
    plt.ylabel('Values', fontsize=14, weight='bold')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Save the plot to a file
    filepath = "../data/analytics_data/"+ encoder_model +"/"+ parameter+ "/crosstext/"+title.replace(' ','') + ".png"
    
    plt.savefig(filepath)
    plt.close()  # Close the plot to avoid overlap issues in the loop


/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_82386/1644806966.py:13: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=categories, y=metric, palette="viridis")  # You can choose other palettes like "pastel", "deep", etc.
/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_82386/1644806966.py:13: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=categories, y=metric, palette="viridis")  # You can choose other palettes like "pastel", "deep", etc.
/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_82386/1644806966.py:13: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `le

In [45]:
stats_results_loaded = {}

for text in holy_texts:
    with open("data/analytics_data/"+ encoder_model +"/"+ parameter+ "/"+text+"_network_stats_results.pkl", "rb") as f:
        stats_results_loaded[text] = pickle.load(f)

In [46]:
def plot_histogram(fig, ax, title, bin_edges, heights):

    bin_widths = [bin_edges[j+1] - bin_edges[j] for j in range(len(bin_edges)-1)]
    
    ax.bar(bin_edges[:-1], heights, width=bin_widths, color='skyblue', edgecolor='black', alpha=0.7)
    ax.set_title(title + ' Histogram')
    ax.set_xlabel('Bins')
    ax.set_ylabel('Frequency')
    # ax.set_xticks(bin_edges)

    number_of_xticks = 5
    selected_ticks = np.linspace(start=bin_edges[0], stop=bin_edges[-1], num=number_of_xticks)
    ax.set_xticks(selected_ticks)

    # Add title and labels



In [47]:
for text in holy_texts:

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    for ax, title in  zip(axs.flat,['Degree', 'Betweenness','Closeness','Eigenvector']):

        bin_edges = stats_results_loaded[text][title + ' Centrality Histogram Bins']
        heights = stats_results_loaded[text][title + ' Centrality Histogram Heights']
        plot_histogram(fig, ax, title, bin_edges, heights)

    plt.tight_layout()
    plt.show()

    filepath = "data/analytics_data/"+ encoder_model +"/"+ parameter+ "/histograms/"+text+ "_Histograms.png"
    
    plt.savefig(filepath)

/var/folders/k1/8px1b4w93kn0627y2fh76chw0000gn/T/ipykernel_32652/1448337773.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
